In [ ]:
#took inspiration from this code https://www.analyticsvidhya.com/blog/2021/04/generate-your-own-dataset-using-gan/

In [ ]:
#importing

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy


import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')

In [ ]:
tf.keras.backend.clear_session() #was having a problem with the training crashing

In [ ]:
from google.colab import drive #importing the data from drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
GENERATE_RES = 5 #3 Generation resolution factor #12 is 1600px
# (1=32, 2=64, 3=96, 4=128, etc.)

#GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
#IMAGE_CHANNELS = 3 

#GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3
PREVIEW_ROWS = 1
PREVIEW_COLS = 1
PREVIEW_MARGIN = 1
GENERATE_SQUARE = 400
IMAGE_SIZE = 400




In [ ]:
# Size vector to generate images from
SEED_SIZE = 100

In [ ]:
# Configuration
DATA_PATH = '/content/drive/MyDrive/london_pics_jpg3'
EPOCHS = 1000
EPOCHS = 1000
BATCH_SIZE = 8 
BUFFER_SIZE = 60000


In [ ]:
print(f"Will generate {GENERATE_SQUARE}px square images.") #how big the output will be 

Will generate 400px square images.


In [ ]:
#calculate the time 
def hms_string(sec_elapsed): 
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [ ]:
training_binary_path = os.path.join(DATA_PATH,
        f'training_data_{GENERATE_SQUARE}_{GENERATE_SQUARE}.npy')


In [ ]:
print(f"Looking for file: {training_binary_path}")

Looking for file: /content/drive/MyDrive/london_pics_jpg3/training_data_400_400.npy


In [ ]:
#if not os.path.isfile(training_binary_path):
 # start = time.time()
  #print("Loading training images...")


In [ ]:
#!ls "/content/drive/My Drive/Colab Notebooks/Romanian Symbols"
 

In [ ]:
#!ls "training_binary_path"

In [ ]:
#store data in binary path so dont have to load in the images every time

if not os.path.isfile(training_binary_path):
    start = time.time()
    print("Loading training images...")
    training_data = []
    faces_path = os.path.join(DATA_PATH)


  
    for filename in tqdm(os.listdir(faces_path)):
        path = os.path.join(faces_path,filename)
        image = Image.open(path).resize((GENERATE_SQUARE,
              GENERATE_SQUARE),Image.ANTIALIAS)
      

        training_data.append(np.asarray(image))
        training_data = np.reshape(training_data,(-1,GENERATE_SQUARE,
              GENERATE_SQUARE,IMAGE_CHANNELS))
        training_data = training_data.astype(np.float32)
        training_data = training_data / 127.5 - 1.



        print("Saving training image binary...")
        training_binary_path = os.path.join(DATA_PATH,
        f'training_data_{GENERATE_SQUARE}_{GENERATE_SQUARE}.npy')

        np.save(training_binary_path,training_data)
        elapsed = time.time()-start
        print (f'Image preprocess time: {hms_string(elapsed)}')
 
else:
  print("Loading previous training pickle...")
  training_data = np.load(training_binary_path)



Loading previous training pickle...


In [ ]:
IMAGE_CHANNELS = 3  # number of color channels (RGB)


In [ ]:
#set up tf dataset

def data_generator():
    for data_point in training_data:
        yield data_point

train_dataset = tf.data.Dataset.from_generator(data_generator, output_types=tf.float32)
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [ ]:
#def data_generator():
    #for data_point in training_data:
      #  yield data_point

#train_dataset = tf.data.Dataset.from_generator(data_generator, output_types=tf.float32) \
    #.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [ ]:
#train_dataset = tf.data.Dataset.from_tensor_slices(training_data) \
   # .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
#build generator 
def build_generator(latent_dim):
    model = keras.Sequential()

    #foundation for 25x25 image
    n_nodes = 128 * 25 * 25
    model.add(keras.layers.Dense(n_nodes, input_dim=latent_dim))
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Reshape((25, 25, 128)))

 # upsample to 50x50
    model.add(keras.layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(keras.layers.LeakyReLU(alpha=0.2))

 # upsample to 100x100
    model.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(keras.layers.LeakyReLU(alpha=0.2))

 # upsample to 200x200
    model.add(keras.layers.Conv2DTranspose(64, (4,4), strides=(2,2), padding='same'))
    model.add(keras.layers.LeakyReLU(alpha=0.2))

 # upsample to 400x400
    model.add(keras.layers.Conv2DTranspose(32, (4,4), strides=(2,2), padding='same'))
    model.add(keras.layers.LeakyReLU(alpha=0.2))

 # upsample to 800x800
   
    #model.add(keras.layers.Conv2DTranspose(8, (4,4), strides=(2,2), padding='same'))
    #model.add(keras.layers.LeakyReLU(alpha=0.2))

 # upsample to 1600x1600
    #model.add(keras.layers.Conv2DTranspose(4, (4,4), strides=(2,2), padding='same'))
    #model.add(keras.layers.LeakyReLU(alpha=0.2))

 # output layer
    model.add(keras.layers.Conv2D(3, (3,3), activation='tanh', padding='same'))



    return model


In [ ]:
#build discriinator
def build_discriminator(img_shape, num_channels):
    model = keras.Sequential()

    model.add(keras.layers.Input(shape=img_shape))
    model.add(keras.layers.Reshape((img_shape[0], img_shape[1], img_shape[2], 1)))
    #model.add(keras.layers.Reshape((img_shape[0], img_shape[1], 1)))
    model.add(keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    

    
    
    #model.add(keras.layers.Input(shape=img_shape))
    #model.add(keras.layers.Reshape((img_shape[0], img_shape[1], num_channels, 1)))
    #model.add(keras.layers.Reshape((img_shape[0], img_shape[1], num_channels)))

    #model.add(keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same'))
    #model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Flatten())
    #model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.add(keras.layers.Dense(3, activation='sigmoid'))
    return model


    


In [ ]:
 #save images in drive
 
 def save_images(epoch, noise, generator):
    image_array = np.full((PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
                           PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
                          255, dtype=np.uint8)

    generated_images = generator.predict(noise)

    generated_images = 0.5 * generated_images + 0.5

    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
            c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
            image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] \
                = generated_images[image_count] * 255
            image_count += 1

    output_path = os.path.join(DATA_PATH, 'output28')
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    filename = os.path.join(output_path, f'train-{epoch}.png')
    im = Image.fromarray(image_array)
    im.save(filename)





In [ ]:
#generate images

def generate_images(generator, seed, save_path):
    images = generator(seed, training=False)
    images = (images * 0.5 + 0.5) * 255
    images = tf.cast(images, tf.uint8)
    for i in range(images.shape[0]):
        img = keras.preprocessing.image.array_to_img(images[i])
        img.save(f"{save_path}/generated_image_{i}.png")


In [ ]:
generator = build_generator(SEED_SIZE)
generator.summary()

# Create the discriminator
image_shape = (IMAGE_SIZE, IMAGE_CHANNELS)
#discriminator = build_discriminator(img_shape=(1600, 1600, 3))
#image_shape = (1600, 1600)
num_channels = 3
image_shape = (400, 400, num_channels)
discriminator = build_discriminator(num_channels=num_channels, img_shape=image_shape)






discriminator.summary()

# Compile the discriminator
discriminator.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80000)             8080000   
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 80000)             0         
                                                                 
 reshape_4 (Reshape)         (None, 25, 25, 128)       0         
                                                                 
 conv2d_transpose_8 (Conv2DT  (None, 50, 50, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 50, 50, 256)       0         
                                                                 
 conv2d_transpose_9 (Conv2DT  (None, 100, 100, 128)    524416    
 ranspose)                                            

In [ ]:
#compile the combined model
z = keras.layers.Input(shape=(SEED_SIZE,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)
combined = keras.models.Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002 , beta_1=0.5))


In [ ]:
#had some problems with the output layer not matching the inout - used this to debug
noise = tf.random.normal([1, SEED_SIZE])
generated_image = generator(noise, training=False)
print(generated_image.shape)


(1, 400, 400, 3)


In [ ]:
decision = discriminator(generated_image)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    total_loss = tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss)
    return total_loss


In [ ]:
 def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)


In [ ]:
LEARNING_RATE = 0.0002
BETA_1 = 0.5
LATENT_DIM = 100

generator_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE, beta_1=BETA_1)


for var in generator.variables:
    if "adam" in var.name:
        generator_optimizer.add_slot(var, "m")
        generator_optimizer.add_slot(var, "v")

for var in discriminator.variables:
    if "adam" in var.name:
        discriminator_optimizer.add_slot(var, "m")
        discriminator_optimizer.add_slot(var, "v")


def train_step(images, generator, discriminator, generator_optimizer, discriminator_optimizer):
    noise = tf.random.normal([BATCH_SIZE, 100])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))





    return gen_loss, disc_loss
    
    


    



In [ ]:
def train(dataset, epochs):
    fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))
    start = time.time()
    #size = (GENERATE_SQUARE, GENERATE_SQUARE)

    generator = build_generator(latent_dim=SEED_SIZE)
    num_channels = 3
    size = (400, 400, num_channels)
    discriminator = build_discriminator(img_shape=(400, 400, num_channels), num_channels=num_channels)

    # ...

    for epoch in range(epochs):
        epoch_start = time.time()
        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            image_batch = tf.image.resize(image_batch, (400, 400))
            t = train_step(image_batch, generator, discriminator, generator_optimizer, discriminator_optimizer)

            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])

        g_loss = sum(gen_loss_list) / len(gen_loss_list)
        d_loss = sum(disc_loss_list) / len(disc_loss_list)
        epoch_elapsed = time.time()-epoch_start

        print(f'Epoch {epoch+1}, gen loss={g_loss}, disc loss={d_loss}, {hms_string(epoch_elapsed)}')


        save_images(epoch, fixed_seed, generator)

    elapsed = time.time()-start
    print(f'Training time: {hms_string(elapsed)}')

train(train_dataset, EPOCHS)
#train(train_dataset, EPOCHS, generator_optimizer, discriminator_optimizer)






Epoch 1, gen loss=0.693126916885376, disc loss=1.3910343647003174, 0:00:09.07
1/1 [==============================] - 0s 126ms/step
Epoch 2, gen loss=0.6978352069854736, disc loss=0.7174233198165894, 0:00:00.13
1/1 [==============================] - 0s 20ms/step
Epoch 3, gen loss=0.7348815202713013, disc loss=0.653320848941803, 0:00:00.13
1/1 [==============================] - 0s 21ms/step
Epoch 4, gen loss=0.8494458198547363, disc loss=0.5580337643623352, 0:00:00.13
1/1 [==============================] - 0s 21ms/step
Epoch 5, gen loss=1.1237845420837402, disc loss=0.3931310474872589, 0:00:00.13
1/1 [==============================] - 0s 20ms/step
Epoch 6, gen loss=1.6959476470947266, disc loss=0.20266549289226532, 0:00:00.13
1/1 [==============================] - 0s 20ms/step
Epoch 7, gen loss=2.6360325813293457, disc loss=0.0744272917509079, 0:00:00.13
1/1 [==============================] - 0s 21ms/step
Epoch 8, gen loss=3.7269957065582275, disc loss=0.024635596200823784, 0:00:00.13
1/